In [1]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

2023-03-03 14:35:55.459564: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-03 14:35:55.459634: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-03 14:35:56.451088: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-03 14:35:56.451210: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-03 14:35:56.451223: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [3]:
import nltk
# from nltk.stem.isri import ISRIStemmer
# nltk.download('punkt')
# nltk.download('isri')

# stemmer = ISRIStemmer()
# text = """
# إِنَّآ أَعْطَيْنَٰكَ ٱلْكَوْثَرَ. فَصَلِّ لِرَبِّكَ وَٱنْحَرْ. إِنَّ شَانِئَكَ هُوَ ٱلْأَبْتَرُ.
# """

text = "إِنَّآ أَعْطَيْنَٰكَ ٱلْكَوْثَرَ"

tokens = [nltk.word_tokenize(i) for i in nltk.sent_tokenize(text)]

In [4]:
# Load the mapped data:
# with open("mapped_data.txt", "r", encoding="utf-8") as file:
#     data = file.readlines()
# data = [line.strip().split() for line in data]
data = tokens

In [5]:
# Split the data into training and testing sets:
# split_ratio = 0.8
# split_index = int(split_ratio * len(data))
train_data = data
test_data = data


In [39]:
# Convert the audio files to spectrograms:
def audio_to_spectrogram(audio_file):
    signal, sample_rate = librosa.load(audio_file)
    spectrogram = librosa.feature.melspectrogram(y=signal)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    return spectrogram

# train_spectrograms = [audio_to_spectrogram(line[0]) for line in train_data]
# test_spectrograms = [audio_to_spectrogram(line[0]) for line in test_data]
train_spectrograms = audio_to_spectrogram(r"audios/ayat1.mp3")
test_spectrograms = audio_to_spectrogram(r"audios/ayat1.mp3")

In [40]:
# Convert the text tokens to numerical labels:
unique_tokens = list(set([token for line in data for token in line[:]]))
token_to_label = {token: i for i, token in enumerate(unique_tokens)}

train_labels = [[token_to_label[token] for token in line[:]] for line in train_data]
test_labels = [[token_to_label[token] for token in line[:]] for line in test_data]

In [41]:
# Pad the spectrograms and labels:
# for spectrogram in train_spectrograms:
#     print(spectrogram.shape)
max_length = max([spectrogram.shape[0] for spectrogram in train_spectrograms])
input_shape = (train_spectrograms[0].shape[0], max_length)

# train_spectrograms = [np.pad(spectrogram, ((0, 0), (0, max_length - spectrogram.shape[1])), "constant") for spectrogram in train_spectrograms]
# test_spectrograms = [np.pad(spectrogram, ((0, 0), (0, max_length - spectrogram.shape[1])), "constant") for spectrogram in test_spectrograms]

train_labels = keras.preprocessing.sequence.pad_sequences(train_labels, padding="post", value=-1)
test_labels = keras.preprocessing.sequence.pad_sequences(test_labels, padding="post", value=-1)
# print(train_labels)

In [42]:
# Define the model architecture:
model = keras.models.Sequential([
    keras.layers.Input(shape=input_shape),
    keras.layers.Conv2D(32, (3, 3), activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(unique_tokens), activation="softmax")
])

2023-03-03 15:43:37.999746: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-03 15:43:37.999811: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-03 15:43:37.999844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-084LSCT): /proc/driver/nvidia/version does not exist


ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 201, 201)